In [3]:
import yfinance as yf

# Download historical data for Apple (AAPL) from Yahoo Finance
data = yf.download("AAPL", start="2010-01-01", end="2023-03-26")

# Print the first 5 rows of the data
print(data.head())


ModuleNotFoundError: No module named 'frozendict'